In [ ]:
def basischanger(c, f):
        """
        allows a change in basis
        
        input:
        c: transformation matrix
        f: operator to be transformed
        """
        return np.einsum("ij,jm,mn->in", c.T, f, c, optimize=True)

In [1]:
class ConstrainedMolecule:
    def __init__(self, geom_file):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
        #getting some old and some new matrices
        self.gamma_alpha = self.id.getDensityMatrix("alpha")
        self.gamma_beta = self.id.getDensityMatrix("beta")
        self.pMatrix = (self.gamma_alpha + self.gamma_beta)/2
        self.transform = eigh(self.pMatrix, self.id.overlap)[1]
        
    
    def fock_Alternator(self, spin):
        "alters a fock operator to follow CUHF"
        assert spin == "alpha" or spin == "beta", f"{spin}: no valid spin"
        f = self.id.displayFockMatrix(spin)
        f_trans = basischanger(self.transform, f)
        coulomb_a = np.einsum("nopq,pq->no", self.id.displayElectronRepulsion(), self.pMatrix, optimize=True)
        exchange = np.einsum("npoq,pq->no", self.id.displayElectronRepulsion(), self.pMatrix, optimize=True)
        closed_shell_type = 0.5*(self.id.displayHamiltonian() + 2*coulomb_a - exchange)
        closed_shell_trans = basischanger(self.transform, closed_shell_type)
        
        # amount of paired electrons
        cc = self.id.beta
        
        # amount of unpaored electrons
        oo = self.id.alpha - self.id.beta
        
        # amount of unoccupied orbitals
        
        # alter first block
        f_trans[:cc + oo, cc + oo:] = closed_shell_type[:cc + oo, cc + oo:]
        
        # alter block in transposed position
        f_trans[cc + oo:, :cc + oo] = closed_shell_type[cc + oo:, :cc + oo]
        
        
        return f_trans
        
    def iteratinator(self):
        self.id.setGuess(self.fock_Alternator("alpha"), "alpha")
        self.id.setGuess(self.fock_Alternator("beta"), "beta")
        return self.id.iterator(iteration=1000)
    
    
    def updatePMatrix(self):
        """updates pMatrix to current self.id densitymatrices"""
        self.pMatrix = (self.id.getDensityMatrix("alpha") + self.id.getDensityMatrix("beta"))